In [44]:
import urllib2, re
from firebase import firebase
import hashlib
from datetime import datetime
from dateutil.parser import parse

In [46]:
src_url = 'http://www.kirklandwa.gov/Kirkland_Calendar.htm'
req = urllib2.Request(src_url)
page = urllib2.urlopen(req).read()

detailLinks = []
# source html file
htmlFile = open('kirklandgov.html','wb')
htmlFile.write(page)
htmlFile.close()

detailRE = re.compile(r'<td class=\'ip-tablecalendar-month-eventtitle\'><a href=\"(.*)\">')
detail = detailRE.findall(page)

for i in range(0, len(detail)):
    if "meeting" in detail[i].lower():
        detailLinks.append(detail[i].replace("&amp;","&"))

titles = []
times = []
dates = []
locations = []
canceled = []

# for i in range(0, len(detailLinks)):
for i in range(0, len(detailLinks)):
    detail_url = detailLinks[i]
    req_detail = urllib2.Request(detail_url)
    page_detail = urllib2.urlopen(req_detail).read()
    
    htmlFile_Detail = open('kirklandDetail.html','wb')
    htmlFile_Detail.write(page_detail)
    htmlFile_Detail.close()
    
    titleRE = re.compile(r'<td id=\"ctl06_ctl47_tdTitleCell\" colspan=\"3\"><span>(.*)</span>')
    # [\s] to match any white space and line break
    timeRE = re.compile(r'<span id=\"ctl06_ctl47_lblEventTime\">(.*)</span>')
    dateRE = re.compile(r'<span id="ctl06_ctl47_lblEventDate">(.*)</span>')
    locationRE = re.compile(r'</span><br>(.*)</td>')
    
    if titleRE.findall(page_detail)[0][-9:] == "Cancelled":
        canceled.append("Yes")
    else:
        titles.append(titleRE.findall(page_detail)[0].replace("\xe2\x80\x94","-"))
        canceled.append("No")
    if len(timeRE.findall(page_detail)) > 0:
        times.append(timeRE.findall(page_detail)[0])
    else:
        times.append("NA")
    if len(dateRE.findall(page_detail)) > 0:

        date = datetime.strptime(dateRE.findall(page_detail)[0],'%B %d, %Y')
        d = parse(date.strftime('%m/%d/%Y %H:%M:%S')+"-0800")
        # set to PST timezone
#         date = pytz.utc.localize(date)
#         date = date.astimezone(timezone('US/Pacific'))
        dates.append(d)
    else:
        dates.append("NA")
    if len(locationRE.findall(page_detail)) > 0:
        locations.append(locationRE.findall(page_detail)[0].replace("<br>"," "))
    else:
        locations.append("NA")
        
# push date to firebase
f = firebase.FirebaseApplication('https://wa-state-calendar.firebaseio.com', None)

for i in range(len(titles)):
    agent = "Kirkland City Council"
    title = titles[i]
    date = dates[i]
    time = times[i]
    location = locations[i]
    cancel = canceled[i]
    if len(time) > 10:
        time = "Flexible/On Appointment"

    # use MD5 string as index to avoid duplicates
    tmpString = agent+title+"Kirkland"+date.strftime('%m/%d/%Y')+time+location;
    MD5String = hashlib.md5(tmpString).hexdigest()

    f.put('meeting/',MD5String,
        {
        "MD5index":MD5String,
         "title": title, 
         "date": date,
         "time": time,
         "location": location,
         "cancelOrNot":cancel,
         "board": "NA",
         "city": "Kirkland",
         "zipcode": 98033,
         "description": "NA",
         "contactName": "NA",
         "contactEmail": "NA",
         "contactPhone": "NA",
         "weblink": "NA",
         "summary": "NA"
    })

# # store data in a file (just for debugging purpose)     
# events = []
# for i in range(len(titles)):
#     item = 'Kirkland City Council '+titles[i]+' '+locations[i]
#     events.append(item)
# events.append(datetime.now().strftime("%I:%M %p on %B %d, %Y"))

# webFile = open('crawler_kirklandgov.html','wb')
# for event in events:
#     webFile.write(event)
#     webFile.write('\n')
# webFile.close()




2017-02-28 00:00:00-08:00
2017-03-01 00:00:00-08:00
2017-03-07 00:00:00-08:00
2017-03-08 00:00:00-08:00
2017-03-08 00:00:00-08:00
2017-03-09 00:00:00-08:00
2017-03-13 00:00:00-08:00
2017-03-14 00:00:00-08:00
2017-03-14 00:00:00-08:00
2017-03-15 00:00:00-08:00
2017-03-15 00:00:00-08:00
2017-03-16 00:00:00-08:00
2017-03-17 00:00:00-08:00
2017-03-20 00:00:00-08:00
2017-03-21 00:00:00-08:00
2017-03-23 00:00:00-08:00
2017-03-23 00:00:00-08:00
2017-03-28 00:00:00-08:00
